In [1]:
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

from pyspark.sql import SparkSession
from operator import add
import string
import json
import re

In [2]:
spark_session = SparkSession\
    .builder\
    .master("spark://192.168.2.167:7077")\
    .appName("G18")\
    .config("spark.executor.cores",2)\
    .config("spark.dynamicAllocation.enabled", True)\
    .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
    .config("spark.shuffle.service.enabled", False)\
    .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
    .config("spark.driver.port",9998)\
    .config("spark.blockManager.port",10005)\
    .getOrCreate()

spark_context = spark_session.sparkContext
spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/16 22:50:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/16 22:50:19 WARN Utils: Service 'sparkDriver' could not bind on port 9998. Attempting port 9999.
22/03/16 22:50:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/16 22:50:21 WARN Utils: Service 'org.apache.spark.network.netty.NettyBlockTransferService' could not bind on port 10005. Attempting port 10006.
22/03/16 22:50:22 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [3]:
!head -1 ~/reddit-final-project/sample

head: cannot open '/home/ubuntu/reddit-final-project/sample' for reading: No such file or directory


In [4]:
_NLTK_STOPWORDS = set([re.sub('\'', '', sw) for sw in stopwords.words('english')])
_NLTK_STEMMER   = PorterStemmer()

In [5]:
lines = spark_context.textFile("hdfs://192.168.2.167:9000/user/ubuntu/data/RC_2011-08")

In [6]:
def json_tuple_mapper(json_string, key, value):
    """
        Converts json string into key-value pair of ID and its associated
        comment. Comments then are lowercased.
    """
    dict=json.loads(json_string)
    id_string=dict[key]
    comment=dict[value].lower().strip()
    return (id_string, comment)

def comment_corrector_mapper(comment):
    allowed_char = string.ascii_letters + string.digits + ' '
    comment = ''.join([letter if letter in allowed_char else '' for letter in comment])
    comment = re.sub(' +', ' ', comment)
    return comment

def NLP_preprocessing_mapper(comment):
    comment_tokenized   = word_tokenize(comment)
    comment_no_stopword = [word for word in comment_tokenized if word not in _NLTK_STOPWORDS]
    comment_stemmed     = [_NLTK_STEMMER.stem(word) for word in comment_no_stopword]
    return comment_stemmed

In [7]:
comments = lines.map(lambda json_str: json_tuple_mapper(json_str,'id','body'))
# comments.take(10)

In [8]:
subreddit = lines.map(lambda json_str: (json.loads(json_str)['subreddit'], 1))
subreddit = subreddit.reduceByKey(add)

In [9]:
filter_threshold = subreddit.filter(lambda tup: tup[1] > 10000)
subreddit_sort = filter_threshold.sortBy(lambda tup: tup[1])
# subreddit_sort.collect()

In [10]:
comments = comments.filter(lambda pair: pair[1]!='[deleted]')
comments = comments.mapValues(comment_corrector_mapper)
comments = comments.mapValues(NLP_preprocessing_mapper)
comments.take(10)

22/03/16 22:53:48 ERROR TaskSchedulerImpl: Lost executor 0 on 192.168.2.167: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


[('c298mtc', ['2', 'would', 'take', 'deal', 'id', 'take', 'deal']),
 ('c298mtg', ['wreck', 'yet']),
 ('c298mth',
  ['think',
   '170k',
   'max',
   'get',
   'anoth',
   'player',
   'book',
   'gibson',
   'would',
   'small',
   'bit',
   'room',
   'maneuv']),
 ('c298mti', ['baton', 'like', 'kill', 'someon', 'taser']),
 ('c298mtl', ['cut', 'break', 'probabl', 'finish', 'watch', 'train', 'day']),
 ('c298mtm', ['100']),
 ('c298mto', ['1000']),
 ('c298mtp',
  ['assum',
   'combin',
   'clopclop',
   'peopl',
   'think',
   'cute',
   'peopl',
   'think',
   'trend',
   'funni',
   'spacehaven',
   'may',
   'prove',
   'assumpt',
   'wrong',
   'though']),
 ('c298mtq', ['wonder', 'smooth', 'top']),
 ('c298mtr', ['clive', 'ownen'])]